0/1ナップサック問題を解いてください。0/1ナップサック問題とは以下のような問題のことです。
* N 個の荷物があり、i(1≦i≦N) 番目の荷物には価値 v_i と重さ w_i が割り当てられている。
* 許容重量 W のナップサックが1つある。
* 重さの和が W 以下となるように荷物の集合を選びナップサックに詰め込むとき、価値の和の最大値を求めよ。ただし、同じ荷物は一度しか選ぶことができない。

https://atcoder.jp/contests/abc032/tasks/abc032_d

Ex1:
In  : 3 10
      15 9
      10 6
      6 4
Out : 16

Ex2:
In  : 30 499887702
      128990795 137274936
      575374246 989051853
      471048785 85168425
      640066776 856699603
      819841327 611065509
      704171581 22345022
      536108301 678298936
      119980848 616908153
      117241527 28801762
      325850062 478675378
      623319578 706900574
      998395208 738510039
      475707585 135746508
      863910036 599020879
      340559411 738084616
      122579234 545330137
      696368935 86797589
      665665204 592749599
      958833732 401229830
      371084424 523386474
      463433600 5310725
      210508742 907821957
      685281136 565237085
      619500108 730556272
      88215377 310581512
      558193168 136966252
      475268130 132739489
      303022740 12425915
      122379996 137199296
      304092766 23505143
Out : 3673016420

Ex3:
In  : 10 2921
      981421680 325
      515936168 845
      17309336 371
      788067075 112
      104855562 96
      494541604 960
      32007355 161
      772339969 581
      55112800 248
      98577050 22
Out : 3657162058

Ex4: 
In  : 10 936447862
      854 810169801
      691 957981784
      294 687140254
      333 932608409
      832 42367415
      642 727293784
      139 870916042
      101 685539955
      853 243593312
      369 977358410
Out : 1686


In [12]:
import itertools 

def init():
  N, W = map(int, input().split())
  return N, W

def init_1(N):
  Va, Wa = [], []
  Vb, Wb = [], []
  for i in range(N):
    v, w = map(int, input().split())
    if i <= N//2:
      Va.append(v)
      Wa.append(w)
    else:
      Vb.append(v)
      Wb.append(w)
  return Va, Wa, Vb, Wb

def init_23(N):
  Vl, Wl = [], []
  for _ in range(N):
    v, w = map(int, input().split())
    Vl.append(v)
    Wl.append(w)
  return Vl, Wl

# N ≤ 30 の時
# 何の制約もなくてしんどいので
def condition_1(N, W, Va, Wa, Vb, Wb):
  # a の組み合わせ
  p = list(itertools.product(range(2), repeat=len(Va)))
  a = []
  for _p in p:
    tmp_v, tmp_w = 0, 0
    for i in range(len(_p)):
      if _p[i] == 1:
        tmp_v += Va[i]
        tmp_w += Wa[i]
    if tmp_w <= W:
      a.append([tmp_v, tmp_w])
  a.sort(key=lambda x: (x[0], -x[1]))
  maxva = -1
  A = []
  for va, wa in a:
    if va <= maxva:
      continue
    else:
      A.append((va, wa))
      maxva = va

  # b の組み合わせ
  p = list(itertools.product(range(2), repeat=len(Vb)))
  b = []
  for _p in p:
    tmp_v, tmp_w = 0, 0
    for i in range(len(_p)):
      if _p[i] == 1:
        tmp_v += Vb[i]
        tmp_w += Wb[i]
    if tmp_w <= W:
      b.append([tmp_v, tmp_w])

  ans = []
  # a と b をマージ
  for vb, wb in b:
    l = 0
    r = len(A)
    while r-l > 1:
      m = (r+l)//2
      if wb+A[m][1] <= W:
        l = m
      else:
        r = m
    ans.append(A[l][0]+vb)
  return max(ans)

# 0 ≤ 重量 ≤ 1e3 の時
def condition_2(N, W, Vl, Wl):
  # 各要素は合計価値，indexは合計重量
  grid = [-1 for _ in range(W+1)]
  grid[0] = 0
  for i in range(N):
    crt_w = Wl[i]
    crt_v = Vl[i]
    # 重量を逆順で走査することで，メモリ消費を抑えられる
    for j in reversed(range(W+1)):
      if j+crt_w <= W and grid[j]!=-1:
        grid[j+crt_w] = max(grid[j+crt_w], grid[j]+crt_v)
  return max(grid)

# 0 ≤ 価値 ≤ 1e3 の時
# 合計価値の最大値が決まっているので，それに合わせて　grid を作成する
def condition_3(N, W, Vl, Wl):
  # 各要素は合計重量，indexは合計価値
  grid = [float("inf") for _ in range(1e3*N+1)]
  grid[0] = 0
  for i in range(N):
    crt_w = Wl[i]
    crt_v = Vl[i]
    for j in reversed(range(1e3*N+1)):
      if j+crt_v <= 1e3*N+1 and grid[j]!=float["inf"]:
        grid[j+crt_v] = min(grid[j+crt_v], grid[j]+crt_w)
  # 完成した　gird　を逆順に走査して，一番最初に W　以下だったものが最大価値
  for i in reversed(range(1e3*N+1)):
    if grid[i] <= W:
      ans = grid[i]
      break
  return ans

def dp_knapsack():
  N, W = init()
  if N <= 30:
    Va, Wa, Vb, Wb = init_1(N)
    ans = condition_1(N, W, Va, Wa, Vb, Wb)
  else:
    Vl, Wl = init_23(N)
    if max(Wl) <= 1e3:
      ans = condition_2(N, W, Vl, Wl)
    elif max(Vl) <= 1e3:
      ans = condition_3(N, W, Vl, Wl)
  return ans

if __name__ == "__main__":
  ans = dp_knapsack()
  print(ans)

16


In [6]:

n, W = map(int, input().split())

if n <= 30:
  va = []
  wa = []
  vb = []
  wb = []
  for i in range(n):
    vv, ww = map(int, input().split())
    if i <= n//2:
      va.append(vv)
      wa.append(ww)
    else:
      vb.append(vv)
      wb.append(ww)

  from itertools import product

  p = list(product(range(2),repeat=len(va)))
  a = []
  for x in p:
    tmpv = 0
    tmpw = 0
    for i in range(len(va)):
      if x[i] == 1:
        tmpv += va[i]
        tmpw += wa[i]
    if tmpw <= W:
      a.append((tmpw, tmpv))
  a.sort(key=lambda x:(x[0], -x[1])) #wでソート
  #それ未満の重みで、それ以上の価値を達成するものがあるものはマージ
  maxv = -1
  A = []
  for w, v in a:
    if v <= maxv:
      continue
    else:
      A.append((w, v))
      maxv = v

  p = list(product(range(2),repeat=len(vb)))
  b = []
  for x in p:
    tmpv = 0
    tmpw = 0
    for i in range(len(vb)):
      if x[i] == 1:
        tmpv += vb[i]
        tmpw += wb[i]
    if tmpw <= W:
      b.append((tmpw, tmpv))
  #bのリストに対して、Aのリストのどの要素とくっつければいいかを考える
  ans = []

  for w, v in b:
    l = 0
    r = len(A)
    while r-l > 1:
      m = (r+l)//2
      if w+A[m][0] <= W:
        l = m
      else:
        r = m
    ans.append(v+A[l][1])

  print(max(ans))

else:
  v = []
  w = []
  for _ in range(n):
    vv, ww = map(int, input().split())
    v.append(vv)
    w.append(ww)
  
  if max(w) <= 1000:
    dp = [-1]*(W+1)
    dp[0] = 0
    for i in range(n):
      ww, vv = w[i], v[i]
      for j in reversed(range(W+1)): #インラインDPのため逆順
        if dp[j] != -1:
          if j + ww <= W:
            dp[j+ww] = max(dp[j+ww], dp[j]+vv)
    print(max(dp))

  
  else:
    inf = 10**20
    dp = [inf]*(1000*n+1)
    dp[0] = 0
    for i in range(n):
      ww, vv = w[i], v[i]
      for j in reversed(range(1000*n+1)): #インラインDPのため逆順
        if dp[j] != inf:
          dp[j+vv] = min(dp[j+vv], dp[j]+ww)

    for j in reversed(range(1000*n+1)):
      if dp[j] <= W:
        print(j)
        break


va:  [15, 10]
wa:  [9, 6]
vb:  [6]
wb:  [4]
p:  [(0, 0), (0, 1), (1, 0), (1, 1)]
a : [(0, 0), (6, 10), (9, 15), (15, 25)]
A : [(0, 0), (6, 10), (9, 15), (15, 25)]
b : [(0, 0), (4, 6)]
25


In [4]:
from itertools import product

va = [15, 10, 100]
p = list(product(range(2),repeat=15))
print()

32768
